In [1]:
#%pip install transformers
#%pip install scipy

In [2]:
import torch, scipy
from transformers import AutoTokenizer, VitsModel
from ipywidgets import widgets
from IPython.display import Audio, display
from ipywidgets import Layout


In [3]:
class Lang2Wav(object):
    def __init__(self, model):
        self.model = VitsModel.from_pretrained(model)
        self.tokenizer = AutoTokenizer.from_pretrained(model)

    def toWav(self, text):
        inputs = self.tokenizer(text, return_tensors="pt")
        with torch.no_grad():
            output = self.model(**inputs).waveform
            sample = output.numpy()[0]
            rate = self.model.config.sampling_rate
            return (sample, rate)

    def save(self, text, filename):
        wav = self.toWav(text)
        scipy.io.wavfile.write(filename, rate=wav[1], data=wav[0])

    def display(self,text=''):
        self.textarea = widgets.Textarea(
            value=text,
            disabled=False,
            layout=Layout(width='80%', height="100px"))
        self.button = widgets.Button(description="Create Sound")
        self.button.on_click(self.button_pressed)
        self.output = widgets.Output(layout=Layout(width='80%', height='200px'))
        display(self.textarea,self.button, self.output)

    def button_save_pressed(self, b):
        with self.output:
            print("Saving to: ", self.savepath.value)
            self.save(self.textarea.value, self.savepath.value)
            print("Saved")

    def button_pressed(self, b):
        with self.output:
            wav = self.toWav(self.textarea.value)
            audio = Audio(data=wav[0], rate = wav[1], autoplay=False)
            self.output.clear_output() # Clear previous output
            self.savepath = widgets.Text(
                value='lang.wav',
                placeholder='',
                description='Save File Path:',
                disabled=False   
            )
            self.button_save = widgets.Button(description="Save")
            self.button_save.on_click(self.button_save_pressed)
            display(audio, self.savepath, self.button_save)
        

    

In [4]:
towav = Lang2Wav("facebook/mms-tts-eng")
towav.display("Hello, how are you today?")

Textarea(value='Hello, how are you today?', layout=Layout(height='100px', width='80%'))

Button(description='Create Sound', style=ButtonStyle())

Output(layout=Layout(height='200px', width='80%'))

In [5]:
towav = Lang2Wav("facebook/mms-tts-vie")
towav.display("VietAI là tổ chức phi lợi nhuận với sứ mệnh ươm mầm tài năng về trí tuệ nhân tạo và xây dựng một cộng đồng các chuyên gia trong lĩnh vực trí tuệ nhân tạo đẳng cấp quốc tế tại Việt Nam.")

Textarea(value='VietAI là tổ chức phi lợi nhuận với sứ mệnh ươm mầm tài năng về trí tuệ nhân tạo và xây dựng m…

Button(description='Create Sound', style=ButtonStyle())

Output(layout=Layout(height='200px', width='80%'))